In [2]:
import os
import numpy as np
import pandas as pd
import slingpy as sp
from genedisco.datasets.features.hgnc_names import HGNCNames
from slingpy.data_access.data_sources.hdf5_tools import HDF5Tools
from slingpy.data_access.data_sources.hdf5_data_source import HDF5DataSource
from slingpy.data_access.data_sources.abstract_data_source import AbstractDataSource

import matplotlib.pyplot as plt
from screen import ScreenData
import scanpy as sc
%matplotlib inline
import matplotlib

In [3]:
save_directory = '/dfs/user/yhr/genedisco/genedisco/datasets/screens/'

### Genedisco/Slingpy dataloader

In [87]:
h5_file = "/dfs/user/yhr/research_assistant/datasets/ground_truth_IFNG.csv"
if not os.path.exists(h5_file):
    d1_file_path = os.path.join(dir_path, "schmidt_ifng_d1.gene_summary.txt")
    df = pd.read_csv(d1_file_path, sep="\t", index_col="id")
    d2_file_path = os.path.join(dir_path, "schmidt_ifng_d2.gene_summary.txt")
    df_d2 = pd.read_csv(d2_file_path, sep="\t", index_col="id")

    df = pd.concat([df, df_d2])
    group_by_row_index = df.groupby(df.index)
    df = group_by_row_index.mean()

    gene_names = df.index.values.tolist()
    name_converter = HGNCNames('/dfs/user/yhr/genedisco/genedisco/datasets/screens/')
    gene_names = name_converter.update_outdated_gene_names(gene_names)
    df.index = gene_names

    # Merge duplicate indices by averaging
    df = df.groupby(df.index).mean()
    gene_names, data = df.index.values.tolist(), df[['pos|lfc']].values.astype(np.float32)


    HDF5Tools.save_h5_file(h5_file,
                           data,
                           "carnevale_2022_{}.h5".format(screen_name),
                           column_names=["log-fold-change"],
                           row_names=gene_names)
data_source = HDF5DataSource(h5_file, duplicate_merge_strategy=sp.MeanMergeStrategy())

In [ ]:
data_source = HDF5DataSource(h5_file, duplicate_merge_strategy=sp.MeanMergeStrategy())

### AI RA dataloader

In [4]:
out_file = "/dfs/user/yhr/research_assistant/datasets/ground_truth_IFNG.csv"
dir_path = "/dfs/user/yhr/genedisco/genedisco/datasets/screens/"

d1_file_path = os.path.join(dir_path, "schmidt_ifng_d1.gene_summary.txt")
df = pd.read_csv(d1_file_path, sep="\t", index_col="id")
d2_file_path = os.path.join(dir_path, "schmidt_ifng_d2.gene_summary.txt")
df_d2 = pd.read_csv(d2_file_path, sep="\t", index_col="id")

df = pd.concat([df, df_d2])
group_by_row_index = df.groupby(df.index)
df = group_by_row_index.mean()

gene_names = df.index.values.tolist()
name_converter = HGNCNames('/dfs/user/yhr/genedisco/genedisco/datasets/screens/')
gene_names = name_converter.update_outdated_gene_names(gene_names)
df.index = gene_names

gene_names, data = df.index.values.tolist(), df[['pos|lfc']].values.astype(np.float32)

# Merge duplicate indices by averaging
df = df.groupby(df.index).mean()
df['pos|lfc'] = df['pos|lfc'].values.astype(np.float32)
df = df.loc[:,['pos|lfc']]
df = df.rename(columns = {'pos|lfc':'log-fold-change'})
df = df.reset_index().rename(columns={'index':'Gene'})

df.to_csv(out_file, index=0)

/dfs/user/yhr/causal/lib/python3.8/site-packages/genedisco/datasets/features/hgnc_names.py:95: DtypeWarning: Columns (31,38) have mixed types.Specify dtype option on import or set low_memory=False.
  previous_mapping = self.get_hgnc_mapping(to_id="symbol", from_id=other_id_name)


In [7]:
file_path = "/dfs/user/yhr/research_assistant/datasets/ground_truth_IFNG.csv"
id_col = 'Gene'
val_col = 'log-fold-change'
bio_taskname = 'IFNG'

screendata = ScreenData(file_path=file_path, id_col=id_col,
       val_col=val_col, bio_taskname=bio_taskname, save=True)
screendata.identify_hits(type_='gaussian', save=True)